## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bincore7_0_x = np.load('x_bc7_0.npy')
bincore7_1_x = np.load('x_bc7_1.npy')
print('x1완료')

bincore7_2_x = np.load('x_bc7_2.npy')
bincore7_3_x = np.load('x_bc7_3.npy')
print('x2완료')

bincore7_0_y = np.load('y_bc7_0.npy')
bincore7_1_y = np.load('y_bc7_1.npy')
bincore7_2_y = np.load('y_bc7_2.npy')
bincore7_3_y = np.load('y_bc7_3.npy')

x_bincore7 = np.concatenate((bincore7_0_x, bincore7_1_x, bincore7_2_x, bincore7_3_x), axis=0)
y_bincore7 = np.concatenate((bincore7_0_y, bincore7_1_y, bincore7_2_y, bincore7_3_y), axis=0)

print(x_bincore7.shape, y_bincore7.shape)

x1완료
x2완료
(333420, 192, 256) (333420, 192, 1)


In [2]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bincore7.shape[0])

x_bincore7 = x_bincore7[p]
y_bincore7 = y_bincore7[p]

print(x_bincore7.shape, y_bincore7.shape)

(333420, 192, 256) (333420, 192, 1)


## (7) 모델

In [3]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 10) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,192, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(288, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습

In [4]:

model1 = models.Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

import time
print('======Training stage======')
starttime = time.time()

model1.fit(x_bincore7,
           y_bincore7,
           epochs = 500,
           batch_size = 32,
           validation_split=0.3,
           callbacks=[early_stopping])

endtime = time.time()

print('걸린시간:', endtime - starttime)
model1.save('gcc7_bin_core_s192_h288_o0123.h5')
print('save 완료')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 192, 256)]        0         
_________________________________________________________________
bidirectional (Bidirectional (None, 192, 576)          1255680   
_________________________________________________________________
time_distributed (TimeDistri (None, 192, 1)            577       
Total params: 1,256,257
Trainable params: 1,256,257
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 233393 samples, validate on 100027 samples
Epoch 1/500
233393/233393 [==============================] - 298s 1ms/sample - loss: 0.0043 - accuracy: 0.9987 - val_loss: 0.0013 - val_accuracy: 0.9996
Epoch 2/500
233393/233393 [==============================] - 296s 1ms/sample - loss: 0.0010 - accuracy: 0.9997 - val_loss: 7.4271e-04 - val_accuracy:

걸린시간: 5592.742951869965
save 완료
